# Neural Style Transfer

## Setup

In [ ]:
%matplotlib inline

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    import h5py

import keras.backend as K
import numpy as np
import scipy

from keras.layers import InputLayer
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, AveragePooling2D
from keras.models import Model, Sequential
from keras.utils.data_utils import get_file
from keras import metrics
from imageio import imwrite
from matplotlib import pyplot as plt
from PIL import Image
from scipy.misc import imsave
from scipy.optimize import fmin_l_bfgs_b

### Limit memory used by Tensorflow

In [ ]:
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

### Define convolutional part of VGG16 model, using average pooling instead of max pooling

In [ ]:
def add_convolutional_layers(model):
    blocks = [
        (2, 64),
        (2, 128),
        (3, 256),
        (3, 512),
        (3, 512)]
    for b in range(len(blocks)):
        block = blocks[b]
        layers = block[0]
        filters = block[1]
        prefix = 'block' + str(b + 1)
        for i in range(layers):
            name = prefix + '_conv' + str(i + 1)
            model.add(Conv2D(filters, (3, 3), activation='relu', padding='same', name=name))
        name = prefix + '_pool'
        model.add(AveragePooling2D((2, 2), strides=(2, 2), name=name))

model = Sequential()
model.add(InputLayer(input_shape=(None, None, 3), name='input'))
add_convolutional_layers(model)
model.summary()

### Load weights

In [ ]:
repo = 'https://github.com/fchollet/deep-learning-models'
weights_url = repo + '/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
local_name = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
weights_path = get_file(local_name, weights_url, cache_subdir='models')
model.load_weights(weights_path)

## Recreating an image

### Load an image for testing

In [ ]:
img = Image.open('data/neural-style/n01558993_9684.png')
img

In [ ]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1, 1, 3))

# Function to subtract imagenet mean and transpose RGB to BGR
preproc = lambda x: (x - vgg_mean)[:, :, :, ::-1]

# Function to transpose BGR to RGB, add imagenet mean, then clip the result
deproc = lambda x,s: np.clip(x.reshape(s)[:, :, :, ::-1] + vgg_mean, 0, 255)

# Preprocess image
img_arr = preproc(np.expand_dims(np.array(img), 0))
shp = img_arr.shape
shp

### Generate initial image

In [ ]:
rand_img = lambda shape: np.random.uniform(0, 1, shape)
x = rand_img(shp)
plt.imshow(x[0]);

### Use solver to minimise loss

In [ ]:
layer = model.get_layer('block5_conv1').output
layer_model = Model(model.input, layer)
targ = K.variable(layer_model.predict(img_arr))

class Evaluator(object):
    def __init__(self, f, shp):
        self.f = f
        self.shp = shp
        
    def loss(self, x):
        loss_, self.grad_values = self.f([x.reshape(self.shp)])
        return loss_.astype(np.float64)

    def grads(self, x):
        return self.grad_values.flatten().astype(np.float64)

def solve_image(eval_obj, niter, x):
    for i in range(niter):
        x, min_val, info = fmin_l_bfgs_b(eval_obj.loss, x.flatten(), fprime=eval_obj.grads, maxfun=20)
        x = np.clip(x, -127, 127)
        print('Loss value at iteration {}: {}'.format(i + 1, min_val))
        if i == niter - 1:
            filename = 'iteration_' + str(i) + '.png'
            imwrite(filename, deproc(x.copy(), shp)[0].astype('uint8'))
            plt.imshow(Image.open(filename))
    return x

iterations=10
loss = K.mean(metrics.mse(layer, targ))
grads = K.gradients(loss, model.input)
fn = K.function([model.input], [loss]+grads)
evaluator = Evaluator(fn, shp)
x = solve_image(evaluator, iterations, x)

## Style extraction

In [ ]:
style = Image.open('data/neural-style/starry_night.png')
style

In [ ]:
style_arr = preproc(style[:,:,:,:3])
shp = style_arr.shape
shp

### Model

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=shp[1:], name='input'))
add_convolutional_layers(model)
model.summary()

### Load weights

In [ ]:
model.load_weights(weights_path)

### Outputs

In [ ]:
outputs = {l.name: l.output for l in model.layers}
layers = [outputs['block{}_conv1'.format(o)] for o in range(1,3)]
layers_model = Model(model.input, layers)
targs = [K.variable(o) for o in layers_model.predict(style_arr)]

### Loss function

In [ ]:
def gram_matrix(x):
    # We want each row to be a channel, and the columns to be flattened x,y locations
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # The dot product of this with its transpose shows the correlation 
    # between each pair of channels
    return K.dot(features, K.transpose(features)) / x.get_shape().num_elements()

def style_loss(x, targ):
    return K.mean(metrics.mse(gram_matrix(x), gram_matrix(targ)))

loss = sum(style_loss(l1[0], l2[0]) for l1,l2 in zip(layers, targs))
grads = K.gradients(loss, model.input)
style_fn = K.function([model.input], [loss]+grads)
evaluator = Evaluator(style_fn, shp)

In [ ]:
x = rand_img(shp)
plt.imshow(x[0]);

In [ ]:
x = scipy.ndimage.filters.gaussian_filter(x, [0,2,2,0])
plt.imshow(x[0]);

In [ ]:
x = solve_image(evaluator, iterations, x)

## Style Transfer

In [ ]:
def plot_arr(arr):
    plt.imshow(deproc(arr, arr.shape)[0].astype('uint8'))

w,h = style.size
src = img_arr[:,:h,:w]
plot_arr(src)

In [ ]:
style_layers = [outputs['block{}_conv2'.format(o)] for o in range(1,6)]
content_name = 'block4_conv2'
content_layer = outputs[content_name]

In [ ]:
style_model = Model(model.input, style_layers)
style_targs = [K.variable(o) for o in style_model.predict(style_arr)]

In [ ]:
content_model = Model(model.input, content_layer)
content_targ = K.variable(content_model.predict(src))

In [ ]:
style_wgts = [0.05,0.2,0.2,0.25,0.3]
loss = sum(style_loss(l1[0], l2[0])*w
           for l1,l2,w in zip(style_layers, style_targs, style_wgts))
loss += K.mean(metrics.mse(content_layer, content_targ) / 10)
grads = K.gradients(loss, model.input)
transfer_fn = K.function([model.input], [loss]+grads)
evaluator = Evaluator(transfer_fn, shp)

### Without gaussian blur

In [ ]:
iterations = 50
x = rand_img(shp)
x = solve_image(evaluator, iterations, x)

### With gaussian blur

In [ ]:
x = rand_img(shp)
x = scipy.ndimage.filters.gaussian_filter(x, [0,2,2,0])
x = solve_image(evaluator, iterations, x)